In [89]:
# from google.colab import drive
# drive.mount('/content/drive')

In [90]:
import numpy as np
import pandas as pd
import re
import nltk

In [91]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
# from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score

In [92]:
# from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
from nltk.corpus import sentiwordnet as swn
from nltk import ngrams, FreqDist
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rachitjain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachitjain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rachitjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rachitjain/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [93]:
data_neg = pd.read_csv('Data/training_negative.csv', encoding='ISO-8859-1')
data_pos = pd.read_csv('Data/training_positive.csv', encoding='ISO-8859-1')

In [94]:
data = data_neg.append(data_pos)
data = data.iloc[:,1:]

In [95]:
data.tail()

,Polarity,Tweet
799995,4,Just woke up. Having no school is the best fee...
799996,4,TheWDB.com - Very cool to hear old Walt interv...
799997,4,Are you ready for your MoJo Makeover? Ask me f...
799998,4,Happy 38th Birthday to my boo of alll time!!! ...
799999,4,happy #charitytuesday @theNSPCC @SparksCharity...


## Sampling Data for Trying out Approaches

In [96]:
df = data.sample(frac=1, random_state=1)
# df = data.iloc[:30000]
df.head()

,Polarity,Tweet
514293,0,i miss nikki nu nu already shes always there ...
142282,0,So I had a dream last night. I remember a sig...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...
649503,0,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now


In [97]:
# To convert the polarity symbols from 4 to 1
df['Polarity'] = np.where(df['Polarity'] == 4, 1, 0)
df.head()

,Polarity,Tweet
514293,0,i miss nikki nu nu already shes always there ...
142282,0,So I had a dream last night. I remember a sig...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...
649503,0,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now


In [98]:
df['Polarity'].value_counts()

1    800000
0    800000
Name: Polarity, dtype: int64

## Pre-Processing Text

In [99]:
def clean_text(tweet):
    tweet = tweet.lower()                                   # Converting to lower case
    tweet = re.sub(r'\b\w+@[^\s]+', ' <MAIL> ', tweet)      # Removing email IDs
    tweet = re.sub(r'@[^\s]+', ' <MENTION> ', tweet)        # Removing mentions
    tweet = re.sub(r'https?:\/[^\s]+', ' <URL> ', tweet)    # Removing URLs
    tweet = re.sub(r'www.[^\s]+', ' <WEBSITE> ', tweet)     # Removing Websites
    tweet = re.sub(r'\b\w+\.com', ' <WEBSITE> ', tweet)      # Removing email IDs
    tweet = re.sub(r'#', ' <HASHTAG> ', tweet)              # Removing hashtags
    tweet = re.sub(r'_', ' ', tweet)                        # Sometimes hashtags are done with _ representing break between two words
    tweet = re.sub(r'\.{2,}', ' ', tweet)                   # Removing sentence separators
    tweet = re.sub(r"[0-9]+",' ', tweet)                    # Removing numbers as they do not indicate sentiment
    tweet = re.sub(r"\bamp\b", ' ', tweet)                  # Removing &amp signs mis-translated
    tweet = re.sub(r"\bquot\b", ' ', tweet)                 # Removing &quot signs mis-translated
    tweet = re.sub(r"\b\w+;[^\s]+\b", ' ', tweet)
    if len(tweet) == 0:
      tweet = 'None'
    return ' '.join(tweet.split())

In [100]:
clean_text("I can't find it")

"i can't find it"

In [101]:
def remove_punc(tweet):
    tweet = re.sub(r"[^\w'\s]+",' ', tweet)                  # Removing punctuations apart from clitic
    return tweet

In [102]:
remove_punc("I can't find it")

"I can't find it"

In [103]:
clean_text("I am &amp rachit1jain@gmail.com n't #doing_exceptionally good hello.com &gt;&gt :D") 

"i am & <MAIL> n't <HASHTAG> doing exceptionally good <WEBSITE> & :d"

In [104]:
def tweet_word_tokenizer(tweet):
    # return word_tokenize(tweet)
    return tweet.split(' ')

In [105]:
# clitics = {
#     "nt": 'not',
#     "ve": 'have',
#     "s": 'is',
#     "m": 'am',
#     "re": 'are',
#     "ll": 'will',
#     'd': 'would',
#     "bout": 'about',
#     'didnt': 'did not',
#     'havent': 'have not',
#     'hasnt': 'has not',
#     'wont': 'will not',
#     'wouldnt': 'will not',
#     'shouldnt': 'should not',
# }

In [106]:
# Handling clitics
def handle_clitics(tweet):
    tweet = re.sub(r"\bwon\'t\b", "will not", tweet)
    tweet = re.sub(r"\bwont \b", "will not", tweet)
    tweet = re.sub(r"\bwouldn't\b", "would not", tweet)
    tweet = re.sub(r"\bwouldnt\b", "would not", tweet)

    tweet = re.sub(r"\bcan\'t\b", "can not", tweet)
    tweet = re.sub(r"\bcant\b", "can not", tweet)

    tweet = re.sub(r"\bdon't\b", "do not", tweet)
    tweet = re.sub(r"\bdont\b", "do not", tweet)
    tweet = re.sub(r"\bdoesn't\b", "does not", tweet)
    tweet = re.sub(r"\bdoesnt\b", "does not", tweet)
    tweet = re.sub(r"\bdidn't\b", "did not", tweet)
    tweet = re.sub(r"\bdidnt\b", "did not", tweet)

    tweet = re.sub(r"\bhasn't\b", "has not", tweet)
    tweet = re.sub(r"\bhasnt\b", "has not", tweet)
    tweet = re.sub(r"\bhaven't\b", "have not", tweet)
    tweet = re.sub(r"\bhavent\b", "have not", tweet)
    tweet = re.sub(r"\bhadn't\b", "had not", tweet)
    tweet = re.sub(r"\bhadnt\b", "had not", tweet)

    tweet = re.sub(r"n\'t", " not", tweet)
    tweet = re.sub(r"\'ve", " have", tweet)
    tweet = re.sub(r"\'re", " are", tweet)
    tweet = re.sub(r"\'s", " is", tweet)
    tweet = re.sub(r"\'m", " am", tweet)
    tweet = re.sub(r"\'re", " are", tweet)
    tweet = re.sub(r"\'ll", " will", tweet)
    tweet = re.sub(r"\'d", " would", tweet)
    
    return ' '.join(tweet.split())

In [107]:
a = handle_clitics("doesnt feel good.Also I can't find it good")
remove_punc(a)

'does not feel good Also I can not find it good'

In [108]:
short_forms = {
    'n': 'and',
    'nu': 'not',
    'no': 'not',
    'ya': 'you',
    'luv': 'love',
    'lol': 'laugh',
    'k': 'okay',
    'na': 'not',
    'ily': 'love',
    'im': 'am',
    'morn': 'morning',
    'nght': 'night',
    'no': 'not',
    'Ill': 'will',
    'shoulda': 'should have',
    'thnks': 'thanks',
    'ty': 'thanks'
    }

In [109]:
def handle_shortforms(tweet):
    temp = ''
    for word in tweet.split():
        if word in short_forms.keys():
            temp = temp + ' ' + short_forms[word]
        else:
            temp = temp + ' ' + word
    return ' '.join(temp.split())

In [110]:
handle_shortforms('I am a good boy shoulda gone')

'I am a good boy should have gone'

In [111]:
## Maintaining only letters within a tweet and removing every other information since not indicative of sentiment
def maintain_letters(tweet):
    tweet = re.sub(r'[^a-zA-Z]', ' ', tweet) 
    return ' '.join(tweet.split())

In [112]:
maintain_letters('i am a good boy. hero is @terohja 909')

'i am a good boy hero is terohja'

In [113]:
# # Emoticons store a lot of information
# def emoticon_translation(tweet):
#     tweet = re.sub(r":\)", " happy ", tweet)
#     tweet = re.sub(r":-\)", " happy ", tweet)
#     tweet = re.sub(r";\)", " happy ", tweet)
#     tweet = re.sub(r":d", " laaugh ", tweet)            # 'aa' in laugh to specify that emoji has a certain significance; also used later in extended versions of ha and lol
#     tweet = re.sub(r";d", " laaugh ", tweet)
#     tweet = re.sub(r"xd", " laaugh ", tweet)
#     tweet = re.sub(r"<3", " love ", tweet)

#     tweet = re.sub(r":\(", " sad ", tweet)
#     tweet = re.sub(r":-\(", " sad ", tweet)
#     tweet = re.sub(r":/", " sad ", tweet)
#     tweet = re.sub(r":\\", " sad ", tweet)
#     tweet = re.sub(r":o", " surprise ", tweet)

#     return tweet

In [114]:
# Emoticons store a lot of information
def emoticon_translation(tweet):
    tweet = re.sub(r":\)", " happy ", tweet)
    tweet = re.sub(r":-\)", " happy ", tweet)
    tweet = re.sub(r";\)", " happy ", tweet)
    tweet = re.sub(r":d", " laaugh ", tweet)            # 'aa' in laugh to specify that emoji has a certain significance; also used later in extended versions of ha and lol
    tweet = re.sub(r";d", " laaugh ", tweet)
    tweet = re.sub(r"xd", " laaugh ", tweet)
    tweet = re.sub(r"<3", " love ", tweet)

    tweet = re.sub(r":\(", " sad ", tweet)
    tweet = re.sub(r":-\(", " sad ", tweet)
    tweet = re.sub(r":/", " sad ", tweet)
    tweet = re.sub(r":\\", " sad ", tweet)
    tweet = re.sub(r":o", " sad ", tweet)           # Suprised might not be the correct representation

    return tweet

In [115]:
# ### TO BE MODIFIED ####
# emoticons_rank = [k for (k_len, k) in reversed(sorted([(len(k), k) for k in emoticons.keys()]))]

In [116]:
# ### TO BE MODIFIED ####
# def emoticons_translation(phrase):
#     for k in emo_info_order:
#         phrase = phrase.replace(k, emo_info[k])
#     return phrase

In [117]:
df['Tweet_regex'] = df['Tweet'].apply(clean_text)
df.head()

,Polarity,Tweet,Tweet_regex
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...
649503,0,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now


In [118]:
# # EDA
# df[df['Polarity'] == 0]['Tweet_regex'].str.contains('<MENTION>').value_counts()

In [119]:
# df[df['Polarity'] == 4]['Tweet_regex'].str.contains('<MENTION>').value_counts()

In [120]:
# # EDA
# df[df['Polarity'] == 0]['Tweet_regex'].str.contains('<WEBSITE>').value_counts()

In [121]:
# # EDA
# df[df['Polarity'] == 4]['Tweet_regex'].str.contains('<WEBSITE>').value_counts()

In [122]:
df['Tweet_emoji'] = df['Tweet_regex'].apply(emoticon_translation)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...
649503,0,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now


In [123]:
df['Tweet_nopunc'] = df['Tweet_emoji'].apply(remove_punc)
# df['Tweet_nopunc'] = df['Tweet_regex'].apply(remove_punc)         # NOT USING EMOJI
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now


In [124]:
df['Tweet_clitics'] = df['Tweet_nopunc'].apply(handle_clitics)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now


In [125]:
df['Tweet_shortforms'] = df['Tweet_clitics'].apply(handle_shortforms)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now


In [126]:
df['Tweet_pure_string'] = df['Tweet_shortforms'].apply(maintain_letters)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now


In [127]:
df['Tweet_token'] = df['Tweet_pure_string'].apply(tweet_word_tokenizer)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa..."
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ..."
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]"
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]"


In [128]:
# # Was absolutely useless to use
# stem = 0
# def stemmer(tweet):
#     global stem
#     stem += 1
#     if stem % 1000:
#       print(stem)
#     porter_stemmer = PorterStemmer()
#     tweet = [porter_stemmer.stem(word) for word in tweet]
#     return tweet

In [129]:
# stemmer(['I','am','playing','making', 'what','I','do'])

In [130]:
# df['Tweet_stem'] = df['Tweet_token'].apply(stemmer)
# df.head()

In [131]:
# def pos_tagger(nltk_tag):
#     if nltk_tag.startswith('J'):
#         return wordnet.ADJ
#     elif nltk_tag.startswith('V'):
#         return wordnet.VERB
#     elif nltk_tag.startswith('N'):
#         return wordnet.NOUN
#     elif nltk_tag.startswith('R'):
#         return wordnet.ADV
#     else:         
#         return None

In [132]:
# #### TO BE MODIFIED ######
# count = 0
# def pos_tagging(tweet):
#     global count
#     count += 1
#     if count % 100 == 0:
#       print(count)
#     # tweet = nltk.pos_tag(tweet) 
#     tweet = nltk.pos_tag([i for i in tweet if i])
#     return tweet

In [133]:
# pos_tagging(['','I','am','good'])

In [134]:
# ### TO BE MODIFIED ################
# def tweet_lemmatizer(tweet):
#     lemmatizer = WordNetLemmatizer()
#     lemmatized = []
#     pos_wordnet = list(map(lambda x: (x[0], pos_tagger(x[1])), tweet))
#     for word, tag in pos_wordnet:
#         if tag is None:
#             lemmatized.append(word)
#         else:       
#             lemmatized.append(lemmatizer.lemmatize(word, tag))
#     lemmatized = ' '.join(lemmatized)
#     lemmatized_sent = ', '.join(lemmatized)
#     # print(list(lemmatized.split()))
#     return list(lemmatized.split())

In [135]:
# df['Tweet_pos'] = df['Tweet_token'].apply(pos_tagging)
# df.head()

In [136]:
# df['Tweet_pos'] = df['Tweet_token'].apply(pos_tagging)
# df.head()

In [137]:
# df['Tweet_lemma'] = df['Tweet_pos'].apply(tweet_lemmatizer)
# df.head()

In [138]:
def sentence_creator(df, col_name, title):
    df[title] = df[col_name].apply(lambda x:' '.join([i for i in x]))
    return df

In [139]:
df = sentence_creator(df, 'Tweet_token', 'Tweet_sent')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now


In [140]:
# wordnet.synsets('hello')

In [141]:
## TO BE MODIFIED #####
def normalisation_words(tweet):
    tweet = tweet.replace(r'([a-z])\1{1,}', r'\1\1')
    # tweet = re.sub(r'(ha){1,}', r'laugh', tweet)
    tweet = re.sub(r'h+a+[ha]+\b', r'laaugh', tweet)     # To give more significance
    # tweet = re.sub(r'(lol){1,}', r'laugh', tweet)
    tweet = re.sub(r'l+o+[lo]+\b', r'laaugh', tweet)

    tweet = re.sub(r'\b([a-z])\1{1,}', r' ', tweet)     # If only repeated letters are left, remove them
    tweet = re.sub(r'\b([a-z])\1{1,}', r' ', tweet)     # If only repeated letters are left, remove them
    tweet = re.sub(r'([a-z])\1{2,}', r'\1\1', tweet)     # If only repeated letters are left, remove them
    tweet = re.sub(r'([a-z])\1{1,}\b', r'\1', tweet)     # If only repeated letters are left, remove them

    # tweet = ' '.join([word if len(wordnet.synsets(word)) > 0 else re.sub(r'([a-z])\1{1,}', r'\1\1', word) for word in tweet.split()])

    tweet = re.sub(r"\b[a-zA-Z]{1}\b", ' ', tweet)        # Removing single letters
    # tweet = re.sub(r'lo+l+o+[^\s]+', r'lol', tweet)
    return tweet.split()

In [142]:
normalisation_words('i dont feel llllooooooooooolllll ')

['dont', 'feel', 'laaugh']

In [143]:
normalisation_words('i awww hahahhhahahahahahahhahaaaaaaaaaaahhhahaha ppeeee lollipop looooooool happppiest day lolllll lll lool bummer get david carr third day laugh')

['aw',
 'laaugh',
 'lollipop',
 'laaugh',
 'happiest',
 'day',
 'laaugh',
 'laaugh',
 'bummer',
 'get',
 'david',
 'car',
 'third',
 'day',
 'laugh']

In [144]:
df['Tweet_normalised'] = df['Tweet_sent'].apply(normalisation_words)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ..."
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y..."
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]"
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]"


In [145]:
df = sentence_creator(df, 'Tweet_normalised', 'Tweet_sent_normal')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ...",mis nikki not not already shes always there wh...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]",it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now


In [146]:
def add_negation(tweet):
    flag = False
    count = 0
    tweet_new = []

    for word in tweet:
        if flag and count < 2:          # Adding prefix to next three words
            word = 'NEG_' + word
            count += 1
        elif word in ['not', 'cannot', 'cant']:     # Others must have already been processed
            flag = True
        tweet_new.append(word)
    return tweet_new

In [147]:
add_negation(['I','not','good','but','bad','hehe'])

['I', 'not', 'NEG_good', 'NEG_but', 'bad', 'hehe']

In [148]:
df['Tweet_normal_negated'] = df['Tweet_normalised'].apply(add_negation)
df.tail()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated
36489,1,@beckybootsx i hope your not drinking alcohol!...,<MENTION> i hope your not drinking alcohol! lol,<MENTION> i hope your not drinking alcohol! lol,MENTION i hope your not drinking alcohol lol,MENTION i hope your not drinking alcohol lol,MENTION i hope your not drinking alcohol laugh,MENTION i hope your not drinking alcohol laugh,"[MENTION, i, hope, your, not, drinking, alcoho...",MENTION i hope your not drinking alcohol laugh,"[MENTION, hope, your, not, drinking, alcohol, ...",MENTION hope your not drinking alcohol laugh,"[MENTION, hope, your, not, NEG_drinking, NEG_a..."
491263,0,That's it!! I can't take it no more!! After su...,that's it!! i can't take it no more!! after su...,that's it!! i can't take it no more!! after su...,that's it i can't take it no more after summ...,that is it i can not take it no more after sum...,that is it i can not take it not more after su...,that is it i can not take it not more after su...,"[that, is, it, i, can, not, take, it, not, mor...",that is it i can not take it not more after su...,"[that, is, it, can, not, take, it, not, more, ...",that is it can not take it not more after summ...,"[that, is, it, can, not, NEG_take, NEG_it, not..."
470924,0,@JonasAustralia i wanted to win! congrats to ...,<MENTION> i wanted to win! congrats to her any...,<MENTION> i wanted to win! congrats to her any...,MENTION i wanted to win congrats to her any...,MENTION i wanted to win congrats to her anyways,MENTION i wanted to win congrats to her anyways,MENTION i wanted to win congrats to her anyways,"[MENTION, i, wanted, to, win, congrats, to, he...",MENTION i wanted to win congrats to her anyways,"[MENTION, wanted, to, win, congrats, to, her, ...",MENTION wanted to win congrats to her anyways,"[MENTION, wanted, to, win, congrats, to, her, ..."
491755,0,Trying to amuse my cousin. It's not working! a...,trying to amuse my cousin. it's not working! a...,trying to amuse my cousin. it's not working! a...,trying to amuse my cousin it's not working a...,trying to amuse my cousin it is not working an...,trying to amuse my cousin it is not working an...,trying to amuse my cousin it is not working an...,"[trying, to, amuse, my, cousin, it, is, not, w...",trying to amuse my cousin it is not working an...,"[trying, to, amuse, my, cousin, it, is, not, w...",trying to amuse my cousin it is not working an...,"[trying, to, amuse, my, cousin, it, is, not, N..."
128037,0,Oh really don't wanna be awake,oh really don't wanna be awake,oh really don't wanna be awake,oh really don't wanna be awake,oh really do not wanna be awake,oh really do not wanna be awake,oh really do not wanna be awake,"[oh, really, do, not, wanna, be, awake]",oh really do not wanna be awake,"[oh, really, do, not, wanna, be, awake]",oh really do not wanna be awake,"[oh, really, do, not, NEG_wanna, NEG_be, awake]"


In [149]:
normalised_words = df.Tweet_sent_normal.str.split(expand=True).stack().value_counts()

In [150]:
more_stop_words = normalised_words.iloc[np.where(normalised_words < 2)].index.tolist()

In [151]:
stop = 0
def stopword_removal(tweet):
    # stoplist = stopwords.words('english')
    # manual_stoplist = ['i', 'nu', 'it', 'u', 'you', 'tweet', 'dm', 'gm', 'gn', 'day', 'and', 'or', 'go', 'get', 'give']
    # stoplist.extend(manual_stoplist)
    # stoplist.remove('not')      # This is an important word
    # stoplist.remove('against')
    # # stoplist.extend(more_stop_words)
    
    # global stop
    # stop += 1
    # if stop % 100 == 0:
    #   print(stop)
    # # stopwords = stopwords.words('english')
    # tweet = [word for word in tweet if word not in stoplist]

    # # if len(tweet) == 0:
    # #     tweet = ['None']
    return tweet

In [152]:
df['Tweet_stopword'] = df['Tweet_normal_negated'].apply(stopword_removal)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated,Tweet_stopword
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ...",mis nikki not not already shes always there wh...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...","[mis, nikki, not, NEG_not, NEG_already, shes, ..."
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...","[MENTION, oh, poor, sickly, you, hugs, hope, y..."
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]","[it, is, raining, again]"
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now]","[MENTION, wish, was, in, la, right, now]"


In [153]:
len(np.where(df.Tweet_stopword.apply(len) >= 30)[0])

787

In [154]:
df.iloc[29231]

Polarity                                                                1
Tweet                   #seb-day #seb-day #seb-day #seb-day #seb-day #...
Tweet_regex             <HASHTAG> seb-day <HASHTAG> seb-day <HASHTAG> ...
Tweet_emoji             <HASHTAG> seb-day <HASHTAG> seb-day <HASHTAG> ...
Tweet_nopunc             HASHTAG  seb day  HASHTAG  seb day  HASHTAG  ...
Tweet_clitics           HASHTAG seb day HASHTAG seb day HASHTAG seb da...
Tweet_shortforms        HASHTAG seb day HASHTAG seb day HASHTAG seb da...
Tweet_pure_string       HASHTAG seb day HASHTAG seb day HASHTAG seb da...
Tweet_token             [HASHTAG, seb, day, HASHTAG, seb, day, HASHTAG...
Tweet_sent              HASHTAG seb day HASHTAG seb day HASHTAG seb da...
Tweet_normalised        [HASHTAG, seb, day, HASHTAG, seb, day, HASHTAG...
Tweet_sent_normal       HASHTAG seb day HASHTAG seb day HASHTAG seb da...
Tweet_normal_negated    [HASHTAG, seb, day, HASHTAG, seb, day, HASHTAG...
Tweet_stopword          [HASHTAG, seb,

In [155]:
df = sentence_creator(df, 'Tweet_stopword', 'Tweet_final_sent')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated,Tweet_stopword,Tweet_final_sent
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ...",mis nikki not not already shes always there wh...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...","[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...","[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]","[it, is, raining, again]",it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now]","[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now


In [156]:
# df.drop(df[df["Tweet_final_sent"] == ''].index, inplace=True)
# df = df.reset_index(drop=True)

In [157]:
df.shape

(1600000, 15)

In [158]:
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated,Tweet_stopword,Tweet_final_sent
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ...",mis nikki not not already shes always there wh...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...","[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...","[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]","[it, is, raining, again]",it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now]","[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now


In [159]:
df.tail()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated,Tweet_stopword,Tweet_final_sent
36489,1,@beckybootsx i hope your not drinking alcohol!...,<MENTION> i hope your not drinking alcohol! lol,<MENTION> i hope your not drinking alcohol! lol,MENTION i hope your not drinking alcohol lol,MENTION i hope your not drinking alcohol lol,MENTION i hope your not drinking alcohol laugh,MENTION i hope your not drinking alcohol laugh,"[MENTION, i, hope, your, not, drinking, alcoho...",MENTION i hope your not drinking alcohol laugh,"[MENTION, hope, your, not, drinking, alcohol, ...",MENTION hope your not drinking alcohol laugh,"[MENTION, hope, your, not, NEG_drinking, NEG_a...","[MENTION, hope, your, not, NEG_drinking, NEG_a...",MENTION hope your not NEG_drinking NEG_alcohol...
491263,0,That's it!! I can't take it no more!! After su...,that's it!! i can't take it no more!! after su...,that's it!! i can't take it no more!! after su...,that's it i can't take it no more after summ...,that is it i can not take it no more after sum...,that is it i can not take it not more after su...,that is it i can not take it not more after su...,"[that, is, it, i, can, not, take, it, not, mor...",that is it i can not take it not more after su...,"[that, is, it, can, not, take, it, not, more, ...",that is it can not take it not more after summ...,"[that, is, it, can, not, NEG_take, NEG_it, not...","[that, is, it, can, not, NEG_take, NEG_it, not...",that is it can not NEG_take NEG_it not more af...
470924,0,@JonasAustralia i wanted to win! congrats to ...,<MENTION> i wanted to win! congrats to her any...,<MENTION> i wanted to win! congrats to her any...,MENTION i wanted to win congrats to her any...,MENTION i wanted to win congrats to her anyways,MENTION i wanted to win congrats to her anyways,MENTION i wanted to win congrats to her anyways,"[MENTION, i, wanted, to, win, congrats, to, he...",MENTION i wanted to win congrats to her anyways,"[MENTION, wanted, to, win, congrats, to, her, ...",MENTION wanted to win congrats to her anyways,"[MENTION, wanted, to, win, congrats, to, her, ...","[MENTION, wanted, to, win, congrats, to, her, ...",MENTION wanted to win congrats to her anyways
491755,0,Trying to amuse my cousin. It's not working! a...,trying to amuse my cousin. it's not working! a...,trying to amuse my cousin. it's not working! a...,trying to amuse my cousin it's not working a...,trying to amuse my cousin it is not working an...,trying to amuse my cousin it is not working an...,trying to amuse my cousin it is not working an...,"[trying, to, amuse, my, cousin, it, is, not, w...",trying to amuse my cousin it is not working an...,"[trying, to, amuse, my, cousin, it, is, not, w...",trying to amuse my cousin it is not working an...,"[trying, to, amuse, my, cousin, it, is, not, N...","[trying, to, amuse, my, cousin, it, is, not, N...",trying to amuse my cousin it is not NEG_workin...
128037,0,Oh really don't wanna be awake,oh really don't wanna be awake,oh really don't wanna be awake,oh really don't wanna be awake,oh really do not wanna be awake,oh really do not wanna be awake,oh really do not wanna be awake,"[oh, really, do, not, wanna, be, awake]",oh really do not wanna be awake,"[oh, really, do, not, wanna, be, awake]",oh really do not wanna be awake,"[oh, really, do, not, NEG_wanna, NEG_be, awake]","[oh, really, do, not, NEG_wanna, NEG_be, awake]",oh really do not NEG_wanna NEG_be awake


In [160]:
# Think if you want to do stratify
X_train, X_test, y_train, y_test = train_test_split(df['Tweet_final_sent'], df['Polarity'], stratify=df['Polarity'], test_size=0.1, random_state=2)

In [161]:
# tfidf_ngrams = TfidfVectorizer(ngram_range=(1,3), min_df=5)
# # ling_stats = LinguisticVectorizer()
# # all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])
# clf = MultinomialNB(alpha=5)

# pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

# pipeline.fit(X_train, y_train)
# y_pred_nb = pipeline.predict(X_test)
# print('F1 Score: ', f1_score(y_test, y_pred_nb))
# sum(y_pred_nb == y_test)/len(y_test)

In [162]:
# tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 3), min_df=1)
# # ling_stats = LinguisticVectorizer()
# # all_features = FeatureUnion([('tfidf', tfidf_ngrams)])

# clf = LogisticRegression(penalty='l2',
#                          solver='saga',
#                          multi_class='multinomial',
#                          tol=1e-5,
#                          n_jobs = -1)

# pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

# pipeline.fit(X_train, y_train)
# y_pred_lr = pipeline.predict(X_test)
# print('F1 Score: ', f1_score(y_test, y_pred_lr))
# sum(y_pred_lr == y_test)/len(y_test)

In [174]:
X_train.shape

(1440000,)

In [164]:
pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+', ngram_range=(1,2))),
                 ('model', LogisticRegression(penalty='l2', max_iter = 200))])
pipe.fit(X_train, y_train)
y_pred_lr = pipe.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

/Users/rachitjain/opt/anaconda3/envs/col772_a2/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


F1 Score:  0.8254590481168294


0.82468125

In [280]:
y_pred_lr = pipe.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

F1 Score:  0.8254590481168294


0.82468125

In [281]:
y_pred_lr.shape

(160000,)

In [176]:
coefs = pd.DataFrame(pipe['model'].coef_, 
                     columns=pipe['vectoriser'].get_feature_names())
coefs = coefs.T.rename(columns={0:'coef'}).sort_values('coef')
coefs

,coef
mis,-15.570541
sad,-14.613006
sadly,-14.273278
rip,-13.396289
poor,-12.888051
...,...
smile,8.946712
neg problem,9.657908
thanks,10.127874
neg bad,10.741498


In [177]:
print(coefs.loc['bad'])
print(coefs.loc['good'])

coef   -8.875133
Name: bad, dtype: float64
coef    7.475829
Name: good, dtype: float64


In [243]:
coef_pos_set = set(coefs.iloc[np.where(coefs['coef'] > 7)].index.tolist())        # 'good' was missing if put 1
coef_neg_set = set(coefs.iloc[np.where(coefs['coef'] < -7)].index.tolist())

In [244]:
# coef_verypos_set = set(coefs.iloc[np.where(coefs['coef'] > 4)].index.tolist())        # 'good' was missing if put 1
# coef_veryneg_set = set(coefs.iloc[np.where(coefs['coef'] < -4)].index.tolist())

In [245]:
# 'good' in coef_verypos_set

In [246]:
# 'not wait' in coef_pos_set

In [247]:
def words_freq(tweet):

  bigram_list = list(ngrams(tweet, 2))
  bigram_string_list = []

  for bigram in bigram_list:
      bigram_string = bigram[0] + ' ' + bigram[1]
      bigram_string_list.append(bigram_string)

#   bigram_string_list = list(ngrams(tweet, 1))

  # num_pos = len(set(tweet).intersection(coef_pos_set))
  # num_neg = len(set(tweet).intersection(coef_neg_set))

  num_pos = len(set(tweet).intersection(coef_pos_set)) + len(set(bigram_string_list).intersection(coef_pos_set))
  num_neg = len(set(tweet).intersection(coef_neg_set)) + len(set(bigram_string_list).intersection(coef_neg_set))
  
#   print(set(bigram_string_list).intersection(coef_pos_set))
#   print(set(tweet).intersection(coef_pos_set))

  # # If there exist positive words in the tweet
  # if num_pos:
  #     for _ in range(num_pos):
  #         tweet.append('POSITIVE')
  # # if num_verypos:
  # #     for num in range(num_verypos):
  # #         tweet.append('POOSITIVE')
  # if num_neg:
  #     for _ in range(num_neg):
  #         tweet.append('NEGATIVE')
  # # if num_veryneg:
  # #     for num in range(num_veryneg):
  # #         tweet.append('NEEGATIVE')

  if num_pos > num_neg:
    return 1
  elif num_neg > num_pos:
    return 0
  else:
    return 0.5

In [248]:
def senti_lexicon(tweet):
  num_pos = len(set(tweet).intersection(coef_pos_set))
  num_neg = len(set(tweet).intersection(coef_neg_set))

  if num_pos > num_neg:
    return 1
  elif num_pos < num_neg:
    return 0
  else:
    return -1

In [249]:
# X_train_token = X_train.apply(tweet_word_tokenizer)

In [250]:
X_test_token = X_test.apply(tweet_word_tokenizer)

In [251]:
# X_train_token

In [282]:
y_pred_lexi = X_test_token.apply(senti_lexicon)
y_pred_lexi.value_counts()

-1    85715
 0    57137
 1    17148
Name: Tweet_final_sent, dtype: int64

In [283]:
y_pred_lr = pd.Series(y_pred_lr, index=X_test.index)
y_pred_lr = np.where(y_pred_lr == 1.0, 1, 0)
y_pred_lr = pd.Series(y_pred_lr, index=X_test.index)
y_pred_lr.value_counts()


1    80713
0    79287
dtype: int64

In [284]:
assert(y_pred_lexi.shape == y_pred_lr.shape)

In [285]:
hehe = 0
for i in range(len(y_pred_lr)):
    if y_pred_lexi.iloc[i] != y_pred_lr.iloc[i]:
        hehe += 1
print(hehe)


95776


In [286]:
y_pred_lexi.iloc[3]

-1

In [295]:
y_pred_lr_modified = y_pred_lr.copy()

for i in range(len(y_pred_lexi)):
    if y_pred_lexi.iloc[i] == 1:
        y_pred_lr_modified.iloc[i] = 1
        # if y_pred_lr.iloc[i] != y_pred_lexi.iloc[i]:
            # print('True')
    elif y_pred_lexi.iloc[i] == 0:
        # if y_pred_lr.iloc[i] != y_pred_lexi.iloc[i]:
            # print('True')
        y_pred_lr_modified.iloc[i] = 0

In [296]:
y_pred_lr.shape

(160000,)

In [297]:
y_pred_lr_modified.shape

(160000,)

In [298]:
(y_pred_lr_modified == y_pred_lr).sum()

149939

In [299]:
print('F1 Score: ', f1_score(y_test, y_pred_lr_modified))
sum(y_pred_lr_modified == y_test)/len(y_test)

F1 Score:  0.7908989720421659


0.8004

In [571]:
words_freq(['I','am','a','good','boy', 'cant', 'wait'])

1

In [573]:
# set(['I','am','a','good','boy', 'cant', 'wait']).intersection(coef_neg_set)

In [575]:
df['Tweet_lexicons_polarity'] = df['Tweet_stopword'].apply(words_freq)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated,Tweet_stopword,Tweet_final_sent,Tweet_lexicons,Tweet_final_sent_lexicons,Tweet_lexicons_freq,Tweet_lexicons_polarity
0,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ...",mis nikki not not already shes always there wh...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...","[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...,0.0,0.0
1,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,0.0,0.0
2,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...","[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,0.0,0.0
3,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again, POSITIVE, NEGATIVE, N...","[it, is, raining, again, POSITIVE, NEGATIVE, N...",it is raining again,"[it, is, raining, again, POSITIVE, NEGATIVE, N...",it is raining again POSITIVE NEGATIVE NEGATIVE...,0.0,0.0
4,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now, NEGAT...","[MENTION, wish, was, in, la, right, now, NEGAT...",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now, NEGAT...",MENTION wish was in la right now NEGATIVE NEGA...,0.0,0.0


In [562]:
# df['Tweet_lexicons'] = df['Tweet_stopword'].apply(words_freq)
# df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated,Tweet_stopword,Tweet_final_sent,Tweet_lexicons,Tweet_final_sent_lexicons
0,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ...",mis nikki not not already shes always there wh...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...","[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...
1,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...
2,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...","[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...
3,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again, POSITIVE, NEGATIVE, N...","[it, is, raining, again, POSITIVE, NEGATIVE, N...",it is raining again,"[it, is, raining, again, POSITIVE, NEGATIVE, N...",it is raining again POSITIVE NEGATIVE NEGATIVE...
4,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now, NEGAT...","[MENTION, wish, was, in, la, right, now, NEGAT...",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now, NEGAT...",MENTION wish was in la right now NEGATIVE NEGA...


In [ ]:
# df = sentence_creator(df, 'Tweet_lexicons', 'Tweet_final_sent_lexicons')
# df.head()

In [576]:
df = sentence_creator(df, 'Tweet_lexicons', 'Tweet_final_sent_lexicons')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_sent,Tweet_normalised,Tweet_sent_normal,Tweet_normal_negated,Tweet_stopword,Tweet_final_sent,Tweet_lexicons,Tweet_final_sent_lexicons,Tweet_lexicons_freq,Tweet_lexicons_polarity
0,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki not not already shes always there...,i miss nikki not not already shes always there...,"[i, miss, nikki, not, not, already, shes, alwa...",i miss nikki not not already shes always there...,"[mis, nikki, not, not, already, shes, always, ...",mis nikki not not already shes always there wh...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...","[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...,"[mis, nikki, not, NEG_not, NEG_already, shes, ...",mis nikki not NEG_not NEG_already shes always ...,0.0,0.0
1,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,"[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...,0.0,0.0
2,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you ...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...","[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,"[MENTION, oh, poor, sickly, you, hugs, hope, y...",MENTION oh poor sickly you hugs hope you feel ...,0.0,0.0
3,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again]",it is raining again,"[it, is, raining, again, POSITIVE, NEGATIVE, N...","[it, is, raining, again, POSITIVE, NEGATIVE, N...",it is raining again,"[it, is, raining, again, POSITIVE, NEGATIVE, N...",it is raining again POSITIVE NEGATIVE NEGATIVE...,0.0,0.0
4,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now, NEGAT...","[MENTION, wish, was, in, la, right, now, NEGAT...",MENTION wish was in la right now,"[MENTION, wish, was, in, la, right, now, NEGAT...",MENTION wish was in la right now NEGATIVE NEGA...,0.0,0.0


In [578]:
# # Think if you want to do stratify
# X_train, X_test, y_train, y_test = train_test_split(df[['Tweet_final_sent','Tweet_lexicons_polarity']], df['Polarity'], stratify=df['Polarity'], test_size=0.1, random_state=2)

In [564]:
# Think if you want to do stratify
X_train, X_test, y_train, y_test = train_test_split(df[['Tweet_final_sent_lexicons']], df['Polarity'], stratify=df['Polarity'], test_size=0.1, random_state=2)

In [581]:
X_train.shape

(1439991, 2)

In [582]:
y_train.shape

(1439991,)

In [580]:
pipe = Pipeline([('vectoriser', CountVectorizer(token_pattern=r'[a-z]+', ngram_range=(1,2))),
                 ('model', LogisticRegression(penalty='l2'))])
pipe.fit(X_train, y_train)
y_pred_lr = pipe.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

ValueError: Found input variables with inconsistent numbers of samples: [2, 1439991]

In [ ]:
# pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+', ngram_range=(1,2))),
#                  ('model', LogisticRegression(penalty='l2', solver='saga', max_iter=1000, tol=1e-5, n_jobs=-1))])
# pipe.fit(X_train, y_train)
# y_pred_lr = pipe.predict(X_test)
# print('F1 Score: ', f1_score(y_test, y_pred_lr))
# sum(y_pred_lr == y_test)/len(y_test)

In [537]:
# coefs = pd.DataFrame(pipe['model'].coef_, 
#                      columns=pipe['vectoriser'].get_feature_names())
# coefs = coefs.T.rename(columns={0:'coef'}).sort_values('coef')
# coefs

In [538]:
coefs.loc['good']

coef    3.072135
Name: good, dtype: float64

In [539]:
# tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 2), min_df=1)

# clf = LogisticRegression(penalty='l2',
#                          solver='saga',
#                          multi_class='multinomial',
#                          tol=1e-5,
#                          n_jobs = -1,
#                          max_iter = 100)

# pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

# pipeline.fit(X_train, y_train)
# y_pred_lr = pipeline.predict(X_test)
# print('F1 Score: ', f1_score(y_test, y_pred_lr))
# sum(y_pred_lr == y_test)/len(y_test)

In [566]:
tfidf_ngrams = CountVectorizer(ngram_range=(1, 2), min_df=1)

clf = LogisticRegression(penalty='l1',
                         solver='saga',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1,
                         max_iter = 1000)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

/Users/rachitjain/opt/anaconda3/envs/col772_a2/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


F1 Score:  0.8147003518701247


0.8133800836255226

In [567]:
tfidf_ngrams = CountVectorizer(ngram_range=(1, 2), min_df=1)

clf = LogisticRegression(penalty='l1',
                         solver='saga',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1,
                         max_iter = 100)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

KeyboardInterrupt: 

In [307]:
df_curt = df[df['Tweet'].str.contains('good')][['Polarity', 'Tweet', 'Tweet_final_sent_lexicons']]

In [114]:
# y_pred_curt = y_pred_lr[df['Tweet'].str.contains('good').index]

IndexError: index 514293 is out of bounds for axis 0 with size 16000

In [ ]:
# assert(y_pred_curt.shape[0] == df_curt.shape[0])

In [100]:
# df_curt.append(y_pred_curt)
df_curt.to_csv('Data/pos.csv')

In [ ]:
tfidf_ngrams = CountVectorizer(ngram_range=(1,3), min_df=5)
# ling_stats = LinguisticVectorizer()
# all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])
clf = MultinomialNB(alpha=0.6)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_nb = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_nb))
sum(y_pred_nb == y_test)/len(y_test)

F1 Score:  0.7334122169253183


0.7562078584921003

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV

In [195]:
LogisticRegression.get_params().keys()

AttributeError: 'str' object has no attribute '_get_param_names'

In [201]:
# # Create a pipeline
# pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+')),
#                  ('model', LogisticRegression())])
# # Prepare a random search
# param_distributions = {'vectoriser__min_df': np.arange(10, 1000, 10),
#                        'vectoriser__max_df': np.linspace(.2, 1, 40),
#                        'model__penalty': ['l2']}
                       
# r_search = RandomizedSearchCV(estimator=pipe, param_distributions=param_distributions, 
#                               n_iter=100, cv=5, n_jobs=4, random_state=123)
# r_search.fit(X_train, y_train)
# # Save results to a dataframe
# r_search_results = pd.DataFrame(r_search.cv_results_).sort_values(by='rank_test_score')

KeyboardInterrupt: 

In [ ]:
columns = [col for col in r_search_results.columns 
           if re.search(r"split|param_", col)]
r_summary = r_search_results[columns].copy()
r_summary.columns = [re.sub(r'_test_score|param_', '', col) 
                     for col in r_summary.columns]
columns = [col.split('__')[1] if '__' in col else col 
           for col in r_summary.columns ]
r_summary.columns = columns
r_summary.head()

,min_df,max_df,penalty,split0,split1,split2,split3,split4
7,50,0.958974,l2,0.757778,0.759375,0.767986,0.763750,0.766319
29,40,0.425641,l2,0.756528,0.759236,0.764514,0.760347,0.762569
15,140,0.753846,l2,0.753472,0.755000,0.761875,0.758403,0.759236
28,140,0.876923,l2,0.753472,0.755000,0.761875,0.758403,0.759236
22,170,0.917949,l2,0.753264,0.754375,0.762500,0.757986,0.759097


In [82]:
def load_sent_word_net():
    sent_scores = collections.defaultdict(list)

    with open("../content/drive/MyDrive/COL772_A2/SentiWordNet_3.0.0.txt","r") as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

        for line in reader:
            if line[0].startswith("#"):
                continue
            if len(line) == 1:
                continue
            POS, ID, PosScore, NegScore, SynsetTerms, Glos = line
            if len(POS) == 0 or len(ID) == 0:
                continue
            for term in SynsetTerms.split(" "):
                term = term.split('#')[0]
                # print(term)
                term = term.replace("-", " ").replace("_", " ")
                key = "%s/%s" % (POS, term)
                # print(key)
                sent_scores[key].append((float(PosScore), float(NegScore)))
                # print(sent_scores)
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value, axis=0)

        return sent_scores


sent_word_net = load_sent_word_net()

In [83]:
class LinguisticVectorizer(BaseEstimator):

    def get_feature_names(self):
        return np.array(['sent_pos', 'sent_neg', 'nouns', 'adjectives', 'verbs', 'adverbs'])

    def fit(self, documents, y=None):
        return self

    def _get_sentiments(self, d):
        sent = tuple(d.split())
        tagged = nltk.pos_tag(sent)

        pos_vals = []
        neg_vals = []

        nouns = 0.
        adjectives = 0.
        verbs = 0.
        adverbs = 0.

        i = 0
        for w, t in tagged:

            p, n = 0, 0
            sent_pos_type = None
            if t.startswith("NN"):
                #noun
                sent_pos_type = "n"
                nouns += 1
            elif t.startswith("JJ"):
                #adjective
                sent_pos_type = "a"
                adjectives += 1
            elif t.startswith("VB"):
                #verb
                sent_pos_type = "v"
                verbs += 1
            elif t.startswith("RB"):
                #adverb
                sent_pos_type = "r"
                adverbs += 1
            else:
                sent_pos_type = "Nan"

                i += 1
                l = len(sent) - i

                if l == 0:
                    l = 1
                else:
                    pass

            if sent_pos_type is not None:

                sent_word = "%s/%s" % (sent_pos_type, w)

                if sent_word in sent_word_net:
                    p, n = sent_word_net[sent_word]
                elif sent_word == "Nan":
                    p, n = 0, 0

                pos_vals.append(p)
                neg_vals.append(n)

        if i == 0:
            l = len(sent)
        else:
            pass

        avg_pos_val = np.mean(pos_vals)
        avg_neg_val = np.mean(neg_vals)

        return [avg_pos_val, avg_neg_val, nouns / l, adjectives / l, verbs / l, adverbs / l]

    # print(_get_sentiments('This be fantastic'))

    def transform(self, documents):
        pos_val, neg_val, nouns, adjectives, verbs, adverbs = np.array([self._get_sentiments(d) for d in documents]).T
        result = np.array([pos_val, neg_val, nouns, adjectives, verbs, adverbs]).T

        return result

In [84]:
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV

In [85]:
# Define functions
def create_baseline_models():
    """Create list of baseline models."""
    models = []
    models.append(('log', LogisticRegression(random_state=123, 
                                             max_iter=1000)))
    models.append(('sgd', SGDClassifier(random_state=123)))
    models.append(('mnb', MultinomialNB()))
    return models

def assess(X, y, models, cv=5, scoring=['roc_auc', 
                                        'accuracy', 
                                        'f1']):
    """Provide summary of cross validation results for models."""
    results = pd.DataFrame()
    for name, model in models:
        result = pd.DataFrame(cross_validate(model, X, y, cv=cv, 
                                             scoring=scoring))
        mean = result.mean().rename('{}_mean'.format)
        std = result.std().rename('{}_std'.format)
        results[name] = pd.concat([mean, std], axis=0)
    return results.sort_index()

In [86]:
models = create_baseline_models()
models

[('log',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=1000,
                     multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False)),
 ('sgd', SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
                l1_ratio=0.15, learning_rate='optimal', loss='hinge',
                max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
                power_t=0.5, random_state=123, shuffle=True, tol=0.001,
                validation_fraction=0.1, verbose=0, warm_start=False)),
 ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

In [87]:
# Preprocess the data
vectoriser = TfidfVectorizer(token_pattern=r'[a-z]+', 
                             stop_words='english', 
                             min_df=30, 
                             max_df=.7)
X_train_simpler = vectoriser.fit_transform(X_train)
# Assess the model
assess(X_train_simpler, y_train, models)

,log,sgd,mnb
fit_time_mean,0.948748,0.125900,0.022557
fit_time_std,0.114444,0.005250,0.003963
score_time_mean,0.016027,0.016216,0.022632
score_time_std,0.000369,0.000349,0.004119
test_accuracy_mean,0.746097,0.746458,0.739278
test_accuracy_std,0.003333,0.002156,0.002977
test_f1_mean,0.752671,0.757212,0.744791
test_f1_std,0.002869,0.001636,0.001948
test_roc_auc_mean,0.827126,0.824481,0.820055
test_roc_auc_std,0.003550,0.003068,0.004196


In [88]:
# Create a pipeline
pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+')),
                 ('model', SGDClassifier(random_state=123))])
# Prepare a random search
param_distributions = {'vectoriser__min_df': np.arange(10, 1000, 10),
                       'vectoriser__max_df': np.linspace(.2, 1, 40),
                       'model__loss': ['log', 'hinge']}
r_search = RandomizedSearchCV(estimator=pipe, param_distributions=param_distributions, 
                              n_iter=30, cv=5, n_jobs=-1, random_state=123)
r_search.fit(X_train, y_train)
# Save results to a dataframe
r_search_results = pd.DataFrame(r_search.cv_results_).sort_values(by='rank_test_score')

In [89]:
columns = [col for col in r_search_results.columns 
           if re.search(r"split|param_", col)]
r_summary = r_search_results[columns].copy()
r_summary.columns = [re.sub(r'_test_score|param_', '', col) 
                     for col in r_summary.columns]
columns = [col.split('__')[1] if '__' in col else col 
           for col in r_summary.columns ]
r_summary.columns = columns
r_summary.head()

,min_df,max_df,loss,split0,split1,split2,split3,split4
4,20,0.220513,hinge,0.768056,0.773611,0.774444,0.773819,0.774167
1,100,0.938462,hinge,0.758750,0.761389,0.764236,0.759444,0.759514
11,100,0.528205,log,0.757431,0.762639,0.765139,0.754792,0.757569
18,120,0.241026,hinge,0.750972,0.756319,0.757153,0.754167,0.754306
9,140,0.671795,hinge,0.751181,0.756528,0.757014,0.753958,0.754028


In [90]:
# Create a pipeline
pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+', max_df=.6)),
                 ('model', SGDClassifier(random_state=123, loss='hinge'))])
# Prepare a grid search
param_grid = {'vectoriser__min_df': [30, 90, 150],
              'vectoriser__ngram_range': [(1,1), (1,2)],
              'vectoriser__stop_words': [None, 'english'],
              'model__fit_intercept': [True, False]}
g_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5, n_jobs=-1)
g_search.fit(X_train, y_train)
# Save results to a dataframe
g_search_results = pd.DataFrame(g_search.cv_results_).sort_values(by='rank_test_score')

In [91]:
# columns = [col for col in g_search_results.columns 
#            if re.search(r"split|param_", col)]
# g_summary = g_search_results[columns+['mean_test_score']].copy()
# g_summary.columns = [re.sub(r'_test_score|param_', '', col) 
#                      for col in g_summary.columns]
# columns = [col.split('__')[1] if '__' in col else col 
#            for col in g_summary.columns ]
# g_summary.columns = columns
# g_summary.head()

In [92]:
# # Create a long dataframe
# g_summary_long = pd.melt(g_summary, 
#                          id_vars=['min_df', 
#                                   'ngram_range', 
#                                   'stop_words', 
#                                   'fit_intercept'], 
#                          value_vars=['split0', 
#                                      'split1', 
#                                      'split2', 
#                                      'split3', 
#                                      'split4'])
# g_summary_long.replace({None: 'None'}, inplace=True)
# # Plot performance
# for param in ['ngram_range', 'stop_words', 'fit_intercept']:
#     plt.figure(figsize=(8,4))
#     plt.title(f'Performance by {param}')
#     sns.boxplot(x='value', y=param, data=g_summary_long, orient='h')
#     plt.xlim(.85, .95);

In [93]:
pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+', min_df=30, max_df=.6, ngram_range=(1,2))),
                 ('model', SGDClassifier(random_state=123, loss='hinge'))])
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectoriser',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.6, max_features=None,
                                 min_df=30, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False, token_pat...
                ('model',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                  

In [94]:
coefs = pd.DataFrame(pipe['model'].coef_, 
                     columns=pipe['vectoriser'].get_feature_names())
coefs = coefs.T.rename(columns={0:'coef'}).sort_values('coef')
coefs

,coef
not,-5.554187
miss,-5.223724
sad,-5.031330
wish,-3.460169
sorri,-3.267135
...,...
happi,2.223506
love,2.342490
good,2.426037
thank,2.572014


In [221]:
coef_pos_set = set(coefs.iloc[np.where(coefs['coef'] > 1)].index.tolist())
coef_neg_set = set(coefs.iloc[np.where(coefs['coef'] < -1)].index.tolist())

In [223]:
def words_freq(tweet):
  num_pos = len(set(tweet).intersection(coef_pos_set))
  num_neg = len(set(tweet).intersection(coef_neg_set))
  
  # If there exist positive words in the tweet
  if num_pos:
      for num in range(num_pos):
          tweet.append('POSITIVE')
  if num_neg:
      for num in range(num_neg):
          tweet.append('NEGATIVE')
  return tweet

In [224]:
# df['coef_pos'] = df['Twitter_final_sent'].str.contains('').value_counts()

In [225]:
df['Tweet_lexicons'] = df['Tweet_stopword'].apply(words_freq)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_final_sent,Tweet_sent,Tweet_normalised,Tweet_stopword,Tweet_lexicons
0,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...",miss nikki nu nu already shes always there whe...,i miss nikki nu nu already shes always there w...,"[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, t..."
1,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so had dream last night remember sign which cl...,so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ..."
2,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[MENTION, ohh, poor, sickly, you, hugs, hope, ..."
3,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,it is raining again,"[it, is, raining, again]","[it, is, raining, again]","[it, is, raining, again]"
4,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish was in la right now,MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]","[MENTION, wish, was, in, la, right, now, NEGAT...","[MENTION, wish, was, in, la, right, now, NEGAT..."


In [226]:
df = make_sentences(df, 'Tweet_lexicons', 'Tweet_final_sent_lexicons')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_final_sent,Tweet_sent,Tweet_normalised,Tweet_stopword,Tweet_lexicons,Tweet_final_sent_lexicons
0,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...",miss nikki nu nu already shes always there whe...,i miss nikki nu nu already shes always there w...,"[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, t...",miss nikki nu nu already shes always there whe...
1,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...",so had dream last night remember sign which cl...,so i had a dream last night i remember a sign ...,"[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...","[so, had, dream, last, night, remember, sign, ...",so had dream last night remember sign which cl...
2,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[MENTION, ohh, poor, sickly, you, hugs, hope, ...",MENTION ohh poor sickly you hugs hope you feel...
3,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]",it is raining again,it is raining again,"[it, is, raining, again]","[it, is, raining, again]","[it, is, raining, again]",it is raining again
4,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]",MENTION wish was in la right now,MENTION wish i was in la right now,"[MENTION, wish, was, in, la, right, now]","[MENTION, wish, was, in, la, right, now, NEGAT...","[MENTION, wish, was, in, la, right, now, NEGAT...",MENTION wish was in la right now NEGATIVE


In [227]:
# Think if you want to do stratify
X_train, X_test, y_train, y_test = train_test_split(df['Tweet_final_sent_lexicons'], df['Polarity'], stratify=df['Polarity'], test_size=0.1, random_state=2)

In [ ]:
print(hello)

In [ ]:
train_pred = pipe.predict(X_train)
print(classification_report(train_pred, 
                            y_train, 
                            target_names=['negative', 'positive']))

In [ ]:
test_pred = pipe.predict(X_test)
print(classification_report(test_pred, 
                            y_test, 
                            target_names=['negative', 'positive']))

              precision    recall  f1-score   support

    negative       0.78      0.80      0.79     77463
    positive       0.81      0.78      0.80     82537

    accuracy                           0.79    160000
   macro avg       0.79      0.79      0.79    160000
weighted avg       0.79      0.79      0.79    160000



In [ ]:
for i in range(10):
    lead = X_test.sample(1)
    %timeit pipe.predict(lead)

1000 loops, best of 5: 674 µs per loop
1000 loops, best of 5: 680 µs per loop
1000 loops, best of 5: 669 µs per loop
1000 loops, best of 5: 677 µs per loop
1000 loops, best of 5: 688 µs per loop
1000 loops, best of 5: 653 µs per loop
1000 loops, best of 5: 697 µs per loop
1000 loops, best of 5: 672 µs per loop
1000 loops, best of 5: 664 µs per loop
1000 loops, best of 5: 671 µs per loop


In [ ]:
pos_words = coefs[coefs['coef']>0].index.tolist()
neg_words = coefs[coefs['coef']<0].index.tolist()

In [ ]:
pos_words_top = coefs[coefs['coef']>1].index.tolist()
neg_words_top = coefs[coefs['coef']<-1].index.tolist()

In [ ]:
import pickle
pickle.dump(pos_words_top, open('/content/drive/MyDrive/COL772_A2/pos_words.txt', 'wb'))
pickle.dump(neg_words_top, open('/content/drive/MyDrive/COL772_A2/neg_words.txt', 'wb'))

In [ ]:
# tfidf_ngrams = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
# all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])
# clf = MultinomialNB(alpha=5)

# pipeline = Pipeline([('all', all_features), ('clf', clf)])

# pipeline.fit(X_train, y_train)

In [95]:
tfidf_ngrams = TfidfVectorizer(ngram_range=(1,3))
ling_stats = LinguisticVectorizer()
# all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])
clf = MultinomialNB(alpha=5)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=5, class_prior=None, fit_prior=True))],
         verbose=False)

In [96]:
# pd.DataFrame(pipeline.predict(X_test)).value_counts()

In [97]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
y_pred_self = pipeline.predict(X_test)

In [98]:
print('F1 Score: ', f1_score(y_test, y_pred_self))

F1 Score:  0.774746687451286


In [99]:
sum(y_pred_self == y_test)/len(y_test)

0.78325

In [100]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_self).ravel()
(tp, fp, tn, fn)

(2982, 700, 3284, 1034)

In [101]:
confusion_matrix(y_test, y_pred_self)

array([[3284,  700],
       [1034, 2982]])

In [228]:
tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])

clf = LogisticRegression(penalty='l1',
                         solver='saga',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1,
                         max_iter = 1000)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

KeyboardInterrupt: ignored

In [134]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_lr).ravel()
(tp, fp, tn, fn)

(3025, 739, 3245, 991)

In [229]:
tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])

clf = LogisticRegression(penalty='l2',
                         solver='saga',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

F1 Score:  0.789614356624666


0.793375

In [104]:
tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])

clf = LogisticRegression(penalty='elasticnet',
                         solver='saga',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr))
sum(y_pred_lr == y_test)/len(y_test)

ValueError: ignored

In [ ]:
pd.DataFrame(y_test)

In [105]:
# tfidf_ngrams = TfidfVectorizer(ngram_range=(1,3))
# ling_stats = LinguisticVectorizer()
# # all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])
# clf = RandomForestClassifier(max_depth=2, random_state=0)

# pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

# pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=2, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
 

In [106]:
# y_pred_rf = pipeline.predict(X_test)
# print('F1 Score: ', f1_score(y_test, y_pred_rf))
# sum(y_pred_rf == y_test)/len(y_test)

F1 Score:  0.698146595883394


0.594875

In [ ]:
res = pd.DataFrame({'Prediction': y_pred_self, 'True':y_test})

In [ ]:
res

In [ ]:
pd.set_option('max_colwidth', 400)

In [ ]:
df.iloc[res[res['Prediction'] != res['True']].index][['Polarity', 'Tweet', 'Tweet_final_sent']]

In [ ]:
df.iloc[res[res['Prediction'] == res['True']].index][['Polarity', 'Tweet', 'Tweet_final_sent']]

In [ ]:
df.iloc[16775]

In [ ]:
# set(stopwords.words('english')).difference(['not', 'very'])

In [ ]:
tfidf_ngrams = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('tfidf', tfidf_ngrams)])
clf = MultinomialNB(alpha=5)

pipeline = Pipeline([('all', all_features), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_nb = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_nb, pos_label=4))
print(sum(y_pred_nb == y_test)/len(y_test))

In [ ]:
len(X_test)

In [ ]:
tfidf_ngrams = TfidfVectorizer(min_df=20, ngram_range=(1, 1))
ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])

clf = LogisticRegression(penalty='l2',
                         solver='lbfgs',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1)

pipeline = Pipeline([('all', all_features), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr, pos_label=4))
sum(y_pred_lr == y_test)/len(y_test)

In [ ]:
tfidf_ngrams.get_feature_names()

In [ ]:
import pickle

In [ ]:
pickle.dump(pipeline, open('/content/drive/MyDrive/COL772_A2/model_25.txt', 'wb'))

In [ ]:
pos_st = df[df['Polarity'] == 4]['Tweet_normalised'].apply(str.split).sum()
neg_st = df[df['Polarity'] == 0]['Tweet_normalised'].apply(str.split).sum()

In [ ]:
pos_uni_freq = FreqDist(ngrams(pos_st, 1))
neg_uni_freq = FreqDist(ngrams(neg_st, 1))
pos_bi_freq = FreqDist(ngrams(pos_st, 2))
neg_bi_freq = FreqDist(ngrams(neg_st, 2))
pos_tri_freq = FreqDist(ngrams(pos_st, 3))
neg_tri_freq = FreqDist(ngrams(neg_st, 3))

In [ ]:
pos_uni_top = pos_uni_freq.most_common(1000)
neg_uni_top = neg_uni_freq.most_common(1000)
pos_bi_top = pos_bi_freq.most_common(1000)
neg_bi_top = neg_bi_freq.most_common(1000)
pos_tri_top = pos_tri_freq.most_common(1000)
neg_tri_top = neg_tri_freq.most_common(1000)

In [ ]:
pos_uni_top

[(('get',), 3923),
 (('go',), 3437),
 (('laugh',), 3364),
 (('good',), 3222),
 (('love',), 2983),
 (('day',), 2701),
 (('like',), 2011),
 (('thanks',), 1767),
 (('time',), 1686),
 (('well',), 1591),
 (('u',), 1581),
 (('see',), 1562),
 (('today',), 1525),
 (('know',), 1443),
 (('work',), 1413),
 (('make',), 1411),
 (('one',), 1397),
 (('new',), 1373),
 (('think',), 1369),
 (('great',), 1326),
 (('night',), 1209),
 (('watch',), 1192),
 (('back',), 1185),
 (('look',), 1126),
 (('oh',), 1094),
 (('would',), 1056),
 (('twitter',), 1052),
 (('come',), 1041),
 (('morning',), 1016),
 (('happy',), 981),
 (('hope',), 963),
 (('really',), 915),
 (('fun',), 905),
 (('wait',), 898),
 (('much',), 886),
 (('want',), 876),
 (('say',), 871),
 (('need',), 854),
 (('nice',), 818),
 (('home',), 806),
 (('thank',), 805),
 (('take',), 761),
 (('hey',), 757),
 (('tomorrow',), 750),
 (('still',), 716),
 (('yeah',), 703),
 (('tweet',), 698),
 (('follow',), 698),
 (('yes',), 686),
 (('awesome',), 686),
 (('thi

In [ ]:
def get_top_words(sent_list):
    word_list = []
    for i in range(len(sent_list)):
        word_list.append(sent_list[i][0][0])
    return word_list

In [ ]:
pos_uni_top_words = get_top_words(pos_uni_top)
neg_uni_top_words = get_top_words(neg_uni_top)
pos_bi_top_words = get_top_words(pos_bi_top)
neg_bi_top_words = get_top_words(neg_bi_top)
pos_tri_top_words = get_top_words(pos_tri_top)
neg_tri_top_words = get_top_words(neg_tri_top)

In [ ]:
print(len(pos_uni_top_words))
print(len(neg_uni_top_words))


1000
1000


In [ ]:
uni_top_common = set(set(pos_uni_top_words) & set(neg_uni_top_words))
pos_best_words = list(set(pos_uni_top_words) - uni_top_common)
neg_best_words = list(set(neg_uni_top_words) - uni_top_common)

In [ ]:
uni_top_common_list = list(uni_top_common)

In [ ]:
pos_best_words

['aha',
 'usually',
 'interesting',
 'hilarious',
 'nah',
 'proud',
 'choose',
 'lovin',
 'thanx',
 'promise',
 'draw',
 'excited',
 'thankyou',
 'bath',
 'appreciate',
 'gorgeous',
 'wo',
 'ahead',
 'sexy',
 'wop',
 'j',
 'ai',
 'positive',
 'official',
 'review',
 'heh',
 'quote',
 'surprise',
 'germany',
 'hun',
 'pizza',
 'consider',
 'john',
 'sex',
 'war',
 'perfect',
 'profile',
 'topic',
 'matt',
 'choice',
 'demi',
 'fav',
 'design',
 'jk',
 'rd',
 'shal',
 'anyways',
 'indeed',
 'dany',
 'often',
 'congratulation',
 'paint',
 'usual',
 'etc',
 'hanah',
 'experience',
 'adorable',
 'shin',
 'anniversary',
 'aka',
 'brilliant',
 'sweetie',
 'bake',
 'unles',
 'prepare',
 'direct',
 'count',
 'folk',
 'sky',
 'studio',
 'view',
 'quiet',
 'gift',
 'tweps',
 'everybody',
 'space',
 'color',
 'fresh',
 'fabulous',
 'lake',
 'joe',
 'include',
 'bright',
 'peace',
 'info',
 'detail',
 'chilin',
 'treat',
 'michael',
 'doin',
 'wave',
 'american',
 'tip',
 'drinking',
 'secret',
 'k

In [ ]:
neg_best_words

['delay',
 'completely',
 'cough',
 'disappointed',
 'fell',
 'cancel',
 'thunder',
 'dnt',
 'kate',
 'ew',
 'piss',
 'blow',
 'terrible',
 'uh',
 'shift',
 'lonely',
 'throat',
 'nightmare',
 'fever',
 'however',
 'blood',
 'confuse',
 'stuck',
 'starbucks',
 'to',
 'broken',
 'traffic',
 'trouble',
 'sat',
 'somewhere',
 'unfortunately',
 'screw',
 'ring',
 'ac',
 'shitty',
 'bank',
 'rid',
 'board',
 'entire',
 'couldnt',
 'shes',
 'painful',
 'wasnt',
 'wah',
 'floor',
 'rip',
 'crappy',
 'doctor',
 'mouth',
 'freeze',
 'key',
 'dread',
 'blackberry',
 'block',
 'laundry',
 'anywhere',
 'co',
 'exhaust',
 'burnt',
 'vet',
 'bummer',
 'blah',
 'no',
 'bo',
 'atm',
 'there',
 'san',
 'shut',
 'slept',
 'arm',
 'tummy',
 'accident',
 'left',
 'wat',
 'france',
 'knee',
 'shame',
 'ear',
 'bored',
 'guted',
 'duno',
 'science',
 'sadly',
 'ouch',
 'leg',
 'report',
 'fml',
 'aint',
 'headache',
 'assignment',
 'darn',
 'revision',
 'depress',
 'weight',
 'schedule',
 'doesnt',
 'gah',


In [ ]:
len(pos_best_words)

205

In [ ]:
# tfidf_ngrams = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
# # ling_stats = LinguisticVectorizer()
# all_features = FeatureUnion([('tfidf', tfidf_ngrams), ('pos', pos_best_words), ('neg', neg_best_words)])
# clf = MultinomialNB(alpha=1)

# pipeline = Pipeline([('all', all_features), ('clf', clf)])

# pipeline.fit(X_train, y_train)
# y_pred_nb = pipeline.predict(X_test)
# print('F1 Score: ', f1_score(y_test, y_pred_nb, pos_label=4))
# print(sum(y_pred_nb == y_test)/len(y_test))

In [ ]:
df['Tweet'].iloc[50]

In [ ]:
df['Tweet_sent'].iloc[50]

In [ ]:
data.iloc[786897,:]

In [ ]:
df.iloc[786897]

In [ ]:
def dummy(tweet):
    return tweet

In [ ]:
cv = CountVectorizer(  
                      tokenizer=dummy,
                      preprocessor=dummy,
                      ngram_range=(1,1)
                    )

In [ ]:
print("Hi")

In [ ]:
X = df['Polarity', 'Tweet_final_sent']
X_train, X_test, y_train, y_test = train_test_split(X['Tweet_final_sent'], X['Polarity'], test_size=0.25, random_state=2)

In [ ]:
# X = cv.fit_transform(df['Tweet_lemma']).toarray()

In [ ]:
X

In [ ]:
X.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, df['Polarity'], test_size=0.25, random_state=2)

In [ ]:
# X_train = X[:80000,:]
# X_test = X[80000:,:]
# y_train = df['Polarity'][:80000]
# y_test = df['Polarity'][80000:]

In [ ]:
X_train.shape

In [ ]:
X_train

In [ ]:
def model_run(model, X_train, y_train):
    model.fit(X_train, y_train)

In [ ]:
def model_predict(model, X_test, y_test):
    print('Accuracy is: ', model.score(X_test, y_test)*100)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

In [ ]:
model = MultinomialNB()
model_run(model, X_train, y_train)
model_predict(model, X_test, y_test)

In [ ]:
# model = LogisticRegression()
# model_run(model, X_train, y_train)
# model_predict(model, X_test, y_test)

In [ ]:
model_predict(model, X_test, y_test)

In [ ]:
model = LinearSVC()
model_run(model, X_train, y_train)
model_predict(model, X_test, y_test)

In [ ]:
df.head()

In [ ]:
tfidf_counts = TfidfVectorizer(tokenizer= word_tokenize, # type of tokenization
                               ngram_range=(1,1)) # number of n-grams
tfidf_data = tfidf_counts.fit_transform(df['Tweet_sent'])

In [ ]:
# tfidf_counts = TfidfVectorizer()
# tfidf_data = tfidf_counts.fit_transform(a)

In [ ]:
tfidf_data.shape

In [ ]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_data, df['Polarity'], test_size=0.25, random_state=2)

In [ ]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(y_train_tfidf.shape)
print(y_test_tfidf.shape)

In [ ]:
model = MultinomialNB()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

In [ ]:
model = LinearSVC()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

In [ ]:
# model = LogisticRegression()
# model_run(model, X_train_tfidf, y_train_tfidf)
# model_predict(model, X_test_tfidf, y_test_tfidf)

In [ ]:
def remove_extra_words(tweet):
    tweet = [word for word in tweet if word in uni_top_common_list]
    if len(tweet) == 0:
        tweet = ['None']
    return tweet

In [ ]:
df['Tweet_remove_extra'] = df['Tweet_lemma'].apply(remove_extra_words)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_clean,Tweet_stopword,Tweet_clitics,Tweet_shortforms,Tweet_pos,Tweet_lemma,Tweet_sent,Tweet_remove_extra
514293,0,i miss nikki nu nu already shes always there ...,miss nikki nu nu already shes always there whe...,"[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[(miss, JJ), (nikki, NN), (nu, JJ), (nu, JJ), ...","[miss, nikki, nu, nu, already, shes, always, n...",miss nikki nu nu already shes always need than...,"[miss, already, shes, always, need, thank, xxx]"
142282,0,So I had a dream last night. I remember a sig...,So had dream last night remember sign which cl...,"[So, had, dream, last, night, remember, sign, ...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[(So, RB), (dream, NN), (last, JJ), (night, NN...","[So, dream, last, night, remember, sign, clear...",So dream last night remember sign clearly tell...,"[So, dream, last, night, remember, sign, tell,..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,ohh poor sickly you hugs hope you feel little ...,"[ohh, poor, sickly, you, hugs, hope, you, feel...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[(ohh, JJ), (poor, JJ), (sickly, JJ), (hugs, N...","[ohh, poor, sickly, hug, hope, feel, little, g...",ohh poor sickly hug hope feel little good soon,"[hug, hope, feel, little, good, soon]"
649503,0,it is raining again,it is raining again,"[it, is, raining, again]",[raining],[raining],[raining],"[(raining, VBG)]",[rain],rain,[rain]
610789,0,@MissKeriBaby wish I was in LA right now,wish was in LA right now,"[wish, was, in, LA, right, now]","[wish, LA, right]","[wish, LA, right]","[wish, LA, right]","[(wish, JJ), (LA, NNP), (right, NN)]","[wish, LA, right]",wish LA right,"[wish, LA, right]"


In [ ]:
df = make_sentences(df, 'Tweet_remove_extra', 'Tweet_final_sent')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_clean,Tweet_stopword,Tweet_clitics,Tweet_shortforms,Tweet_pos,Tweet_lemma,Tweet_sent,Tweet_remove_extra,Tweet_final_sent
514293,0,i miss nikki nu nu already shes always there ...,miss nikki nu nu already shes always there whe...,"[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[(miss, JJ), (nikki, NN), (nu, JJ), (nu, JJ), ...","[miss, nikki, nu, nu, already, shes, always, n...",miss nikki nu nu already shes always need than...,"[miss, already, shes, always, need, thank, xxx]",miss already shes always need thank xxx
142282,0,So I had a dream last night. I remember a sig...,So had dream last night remember sign which cl...,"[So, had, dream, last, night, remember, sign, ...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[(So, RB), (dream, NN), (last, JJ), (night, NN...","[So, dream, last, night, remember, sign, clear...",So dream last night remember sign clearly tell...,"[So, dream, last, night, remember, sign, tell,...",So dream last night remember sign tell get job...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,ohh poor sickly you hugs hope you feel little ...,"[ohh, poor, sickly, you, hugs, hope, you, feel...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[(ohh, JJ), (poor, JJ), (sickly, JJ), (hugs, N...","[ohh, poor, sickly, hug, hope, feel, little, g...",ohh poor sickly hug hope feel little good soon,"[hug, hope, feel, little, good, soon]",hug hope feel little good soon
649503,0,it is raining again,it is raining again,"[it, is, raining, again]",[raining],[raining],[raining],"[(raining, VBG)]",[rain],rain,[rain],rain
610789,0,@MissKeriBaby wish I was in LA right now,wish was in LA right now,"[wish, was, in, LA, right, now]","[wish, LA, right]","[wish, LA, right]","[wish, LA, right]","[(wish, JJ), (LA, NNP), (right, NN)]","[wish, LA, right]",wish LA right,"[wish, LA, right]",wish LA right


In [ ]:
tfidf_counts_clean = TfidfVectorizer(tokenizer= word_tokenize, # type of tokenization
                               ngram_range=(1,2)) # number of n-grams
tfidf_data_clean = tfidf_counts_clean.fit_transform(df['Tweet_final_sent'])

In [ ]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_data_clean, df['Polarity'], test_size=0.25, random_state=2)

In [ ]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(y_train_tfidf.shape)
print(y_test_tfidf.shape)

(30000, 69746)
(10000, 69746)
(30000,)
(10000,)


In [ ]:
model = MultinomialNB()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

Accuracy is:  70.34
              precision    recall  f1-score   support

           0       0.71      0.70      0.70      5048
           4       0.70      0.71      0.70      4952

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000



In [ ]:
model = LinearSVC()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

Accuracy is:  69.61
              precision    recall  f1-score   support

           0       0.71      0.66      0.69      5048
           4       0.68      0.73      0.70      4952

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000



In [ ]:
model = LogisticRegression()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

Accuracy is:  71.04
              precision    recall  f1-score   support

           0       0.73      0.69      0.71      5048
           4       0.70      0.74      0.72      4952

    accuracy                           0.71     10000
   macro avg       0.71      0.71      0.71     10000
weighted avg       0.71      0.71      0.71     10000



In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
text_clf = Pipeline([
    ('tfidf',TfidfVectorizer(preprocessor=None,
                             tokenizer=word_tokenize,
                             analyzer='word',
                             stop_words=None,
                             strip_accents=None,
                             lowercase=True,
                             ngram_range=(1,3),
                             min_df=0.0001,
                             max_df=0.9,
                             binary=False,
                             norm='l2',
                             use_idf=1,
                             smooth_idf=1,
                             sublinear_tf=1)),
    ('clf', LogisticRegression(penalty='l2',
                               solver='saga',
                               multi_class='multinomial',
                              tol=1e-5,
                              n_jobs = -1)),
])

In [ ]:

text_clf.fit(X_train,y_train)